# Lab 02 (remixed): Training and Evaluating a Decision Tree

Answer the exercise questions.

**Objectives**: After completing these exercises, you should be able to:

* Identify the components of an ML annotation project
* Clean some data
* Code a decision tree

Written by: Dr. Stephen Wu

References: [Titanic on Kaggle](https://www.kaggle.com/competitions/titanic)

## Setting up the environment and Titanic data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

%pip install anytree
from anytree import Node, RenderTree

# Install scikit-learn if not already installed, and run
%pip install scikit-learn
import sklearn # machine learning algorithms

In [ ]:
# Get the Titanic data (originally from Kaggle)
dl_train_url = "../lab01/train.csv"
dl_test_url = "../lab01/test.csv"

train_data = pd.read_csv(dl_train_url)
test_data = pd.read_csv(dl_test_url)

### Exercise 2.1: Problem setup (GRADED)
Mark the correct answer with an `X`.

1. Eventually, we want to _predict_ whether people `Survived` or not. What kind of an ML problem will this be?

```
    a) Clustering
    b) Classification
    c) Regression
    d) Generation
```

2. What's the difference between `train_data` and `test_data`? '

```
    a) `test_data` has extra variables that `train_data` doesn't
    b) `test_data` lacks the output variable
    c) `test_data` is used first to help the algorithm find patterns on real data
    d) `train_data` is 
```

3. The split between `train_data` and `test_data` in this problem enables you to do what kind of learning?

```
    a) Supervised learning
    b) Unsupervised learning
    c) Reinforcement learning
    d) Representation learning
```


### Exercise 2.2: Code interpretation (GRADED)
Write a comment using `# <WRITE A COMMENT>` at the end of each line. Hint: Check the documentation for [`Dataframe.fillna()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna) and [`Dataframe.drop()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html#pandas.DataFrame.drop).

In [ ]:
# NaN stands for "Not a Number" and is a common way to represent missing data
# What if our machine learning algorithm doesn't know how to handle missing data?
print(train_data.isna().sum())
train_data = train_data.fillna(method='ffill')
test_data = test_data.fillna(method='ffill')

# Some variables aren't useful for prediction, or aren't easy to use
train_data = train_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, errors='ignore')
test_data = test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, errors='ignore')

## Decision Trees (pt 1): Data to test the algorithm
In class, we talked about decision trees. The pseudocode given (from the textbook AIMA 19.3) was as follows:

**function** LEARN-DECISION-TREE(*examples*, _attributes_, _parent\_examples_) **returns** a tree<br>
&nbsp;&nbsp;&nbsp;&nbsp;**if** _examples_ is empty **then return** PLURALITY-VALUE(_parent\_examples_)<br>
&nbsp;&nbsp;&nbsp;&nbsp;**else if** all _examples_ have the same classification **then return** the classification<br>
&nbsp;&nbsp;&nbsp;&nbsp;**else if** _attributes_ is empty **then return** PLURALITY-VALUE(_examples_)<br>
&nbsp;&nbsp;&nbsp;&nbsp;**else**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_A_ = argmax(IMPORTANCE(_a_, _examples_))<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_tree_ = a new decision tree with root test _A_<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**for each** value _v_ of _A_ **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_exs_ = {_e_ : _e_ in _examples_ **and** _e.A_ = _v_}<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_subtree_ = LEARN-DECISION-TREE(_exs_, _attributes_ - _A_, _examples_)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;add a branch to _tree_ with label (_A_ = _v_) and subtree _subtree_<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**return** _tree_


You will write this function in Python!

In [ ]:
# Information Gain is the standard "Importance" for Decision Trees
# Take these for granted for now
def information_gain(attribute, examples):
    return entropy(examples) - remainder(attribute, examples)

def entropy(examples):
    num_survived = len(examples[examples['Survived'] == 1])
    num_died = len(examples[examples['Survived'] == 0])
    total = len(examples)
    if num_survived == 0 or num_died == 0:
        return 0
    p_survived = num_survived / total
    p_died = num_died / total
    return -p_survived * np.log2(p_survived) - p_died * np.log2(p_died)

def remainder(attribute, examples):
    total = len(examples)
    remainder = 0
    for value in examples[attribute].unique():
        exs = examples[examples[attribute] == value]
        remainder += len(exs) / total * entropy(exs)
    return remainder

 

In [ ]:
%pip install anytree
from anytree import AnyNode, RenderTree

root_node = AnyNode(id="root", parent=None, best_attribute="Area")
left_branch = AnyNode(id="L", parent=root_node, best_attribute=None)
right_branch = AnyNode(id="R", parent=root_node, best_attribute="Alt")
rightleft_branch = AnyNode(id="RL", parent=right_branch, best_attribute=None)
rightright_branch = AnyNode(id="RR", parent=right_branch, best_attribute=None)
print(RenderTree(root_node))


## Exercise 2.3: Decision tree implementation (GRADED)
Turn the pseudocode for LEARN-DECISION-TREE into a real Python function, calling the `information_gain()` function defined above.

You also need to define `plurality_value()`, which should return a `0` if there are more 0s left in the `examples`, or a `1` if more 1s are left in the `examples.

In [ ]:
def plurality_value(examples):
    return examples['Survived'].value_counts().idxmax()

def learn_decision_tree(examples, attributes, parent_examples):
    """
    Recursively chooses attributes, defining a decision tree
    that best fits the examples given, based on information 
    gain.

    Args:
       examples (Dataframe): samples of Titanic dataset
       attributes (list): column names that are not used yet
       parent_examples (Dataframe): examples at parent node

    Returns:
        AnyNode: the root of the decision tree
    """

    # print("DEBUG: Starting learn_decision_tree() with\n"
    #         + f"{len(examples)} examples\n"
    #         + f"{len(attributes)} attributes")

    # Case: leaf node with no examples (all in other branch)
    if examples.empty:
        classification = plurality_value(parent_examples)
        leaf_node = AnyNode(
            id="no_examples", 
            label=classification,
            num_examples=0)
        # print("DEBUG: Finishing ", RenderTree(leaf_node))
        return leaf_node

    # Case: leaf node with uniform class
    elif len(examples['Survived'].value_counts()) == 1:
        classification = examples['Survived'].iloc[0]
        leaf_node = AnyNode(
            id="same_class",
            label=classification,
            num_examples=len(examples))
        # print("DEBUG: Finishing ", RenderTree(leaf_node))
        return leaf_node

    # Case: leaf node with no attributes (pick majority class)
    elif len(attributes) == 0:
        classification = plurality_value(examples)
        leaf_node = AnyNode(
            id="no_attributes", 
            label=classification,
            num_examples=len(examples))
        # print("DEBUG: Finishing ", RenderTree(leaf_node))
        return leaf_node

    # Case: non-leaf node
    else:
        # Choose the best attribute to split on (argmax importance)
        best_attribute = None
        best_importance = -1
        for attribute in attributes:
            importance = information_gain(attribute, examples)
            # print(f"DEBUG: tried {attribute}, has importance {importance}")
            if importance > best_importance:
                best_importance = importance
                best_attribute = attribute

        # Create a new decision tree node with the best attribute
        tree = AnyNode(
            id=best_attribute,
            parent=None, # temporary
            num_examples=len(examples))
        # print("DEBUG: Creating ", RenderTree(tree))
        
        # Remove the best attribute so children don't use it
        child_attributes = [a for a in attributes \
            if a != best_attribute]

        # For each value of the best attribute, create a branch
        for value in examples[best_attribute].unique():
            exs = examples[examples[best_attribute] == value]
            subtree = learn_decision_tree(exs, child_attributes, examples)
            subtree.parent = tree
            subtree.parent_value = value

        return tree


In [ ]:
mini_data = train_data.head(50)

parent_examples_A = mini_data[mini_data['Sex'] == 'female']
examples_A = mini_data[
    ((mini_data['Sex'] == 'female') & mini_data['Survived'] == 1)]
examples_B = mini_data[mini_data['Age'] > 40]
examples_C = mini_data[mini_data['Age'] > 90]

testable_attributes = mini_data.columns.unique().to_list()
testable_attributes.remove('Survived')
testable_attributes.remove('PassengerId')
testable_attributes.remove('Fare')

binned_data = train_data.copy()
binned_data['Age'] = pd.cut(binned_data['Age'], bins=5)
print(binned_data['Age'].value_counts())

decision_tree = learn_decision_tree(
    binned_data, testable_attributes, binned_data)
print(RenderTree(decision_tree))


Now, let's test our code on a very simple data set.

In [ ]:
example_data = pd.DataFrame({
    'A1': [1, 1, 0, 1, 1],
    'A2': [0, 0, 1, 1, 1],
    'A3': [0, 1, 0, 1, 0],
    'Survived' : [0, 0, 0, 1, 1]})

print(learn_decision_tree(example_data, example_data, None))

What we're really after, though, is a decision tree that will be learned on the training set. Show what decision tree this `information_gain` importance function will produce, given this set of data?

In [ ]:
output = learn_decision_tree(train_data, train_data.columns[:-1], None)
print(output)